In [1]:
from numpy import *
import sys
import numba # numba : jit 컴파일러
from cplex import *

# Make Problems

In [50]:
def makeProbs(datafile):
    f = open(datafile,'r')
    lines = f.readlines()

    probs = []

    prob = []
    for line in lines:
        line = line.rstrip()

        if line.startswith('-----'):
            probs.append(prob)
            prob = []
        elif line != '':
            prob.append(line)

    return [makeProb(p) for p in probs]
        
def sort_desc_by_a(c, a, sol):
    sorted_idx = sorted(range(len(a)),key=lambda x:a[x], reverse=True)
    new_a = [a[si] for si in sorted_idx]
    new_c = [c[si] for si in sorted_idx]
    new_sol = [sol[si] for si in sorted_idx]
    d = [int(na*0.3) for na in new_a]
    return new_c, new_a, new_sol, d

def makeProb(dat):
    info = []
    for dp in dat[1:5]:
        info.append(float(dp.split()[1]))
    
    item = []
    for d in dat[5:]:
        a = d.split(',')
        a = map(int,a)
        item.append(a)
    
    c = [it[1] for it in item]
    a = [it[2] for it in item]
    sol = [it[3] for it in item]
    sorted_c, sorted_a, sorted_sol, sorted_d = sort_desc_by_a(c,a,sol)
    
    return {
        'n': int(info[0]),
        'b': int(info[1]),
        'optval': int(info[2]),
        'c' : sorted_c, 
        'a' : sorted_a,
        'sol' : sorted_sol,
        'd' : sorted_d
        
    }

In [36]:
probs = makeProbs('./sc/knapPI_1_50_1000.csv')

In [40]:
probs[0]['d']

[298,
 295,
 291,
 286,
 276,
 270,
 268,
 264,
 262,
 256,
 253,
 238,
 224,
 221,
 215,
 214,
 204,
 202,
 193,
 180,
 172,
 158,
 146,
 145,
 129,
 126,
 110,
 97,
 96,
 89,
 83,
 80,
 75,
 74,
 72,
 63,
 59,
 43,
 41,
 36,
 36,
 32,
 29,
 29,
 28,
 21,
 21,
 12,
 8,
 2]

# 1. Binary Knapsack

### 1.1. Recursive Method

In [41]:
def f(r, lmd, c, a):
    pick = c[r]
    if lmd<=0 or r<=0: return 0
    if lmd<a[r]:
        pick=0
    return max(f(r-1,lmd,c,a), pick+f(r-1,lmd-a[r],c,a))


In [42]:
def solve_recursion(prob):
    c = [0]+prob['c']
    a = [0]+prob['a']
    b = prob['b']
    n = prob['n']
    optval = prob['optval']
    myVal = f(n, b, c, a)
    if optval != myVal:
        print 'wrong', myVal, optval

In [43]:
%%timeit
solve_recursion(probs[0])

1 loops, best of 3: 4.85 s per loop


### 1.2. Iterative Method

In [44]:
@numba.jit(nopython=True) # just in time

def f4(r, lmd, c, a):
    k = zeros((lmd+1, r+1))
    
    for i in range(lmd+1):
        for j in range(1,r+1):
            if i<a[j]: pick = 0
            else: pick = c[j]
            k[i][j] = max(k[i][j-1], pick+k[i-a[j]][j-1])
    return k[-1][-1]

In [45]:
def solve_iterative(pr):
    c = [0]+pr['c']
    a = [0]+pr['a']
    b = pr['b']
    n = pr['n']
    optval = pr['optval']
    myVal = f4(n, b, c, a)
    print myVal
    if optval != myVal:
        print 'wrong', myVal, optval

In [46]:
# %%timeit
solve_iterative(probs[0])

8373.0


### 1.3. Cplex

In [47]:
def knpasack_cplex(n,a,b,c):
    cplex = Cplex()
    
    # Set variables
    x = ['x%d' % i for i in range(n)] #variable name
    cplex.variables.add(obj=c, types=['B'] * n, names=x)
    
    # Set constraints
    cplex.linear_constraints.add(lin_expr=[SparsePair(ind=x, val=a)], \
                                 senses=['L'], rhs=[b], names=['knp'])
    
    #Set sense
    cplex.objective.set_sense(cplex.objective.sense.maximize)
    
    #Solve
    cplex.solve()
    
    obj_val = cplex.solution.get_objective_value()
    x_res = [(cplex.solution.get_values(x[i]), x[i]) for i in range(n) \
             if cplex.solution.get_values(x[i]) > 0]
    return obj_val #, x_res

In [48]:
def solve_knapsack_cplex(p):
    n = p['n']
    a = p['a']
    b = p['b']
    c = p['c']
    return knpasack_cplex(n,a,b,c), p['optval']

In [49]:
solve_knapsack_cplex(probs[50])

Found incumbent of value 0.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
Reduced MIP has 1 rows, 50 columns, and 50 nonzeros.
Reduced MIP has 50 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.07 ticks)
Probing time = 0.00 sec. (0.01 ticks)
Tried aggregator 1 time.
Reduced MIP has 1 rows, 50 columns, and 50 nonzeros.
Reduced MIP has 50 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.08 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 4 threads.
Root relaxation solution time = 0.00 sec. (0.02 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0+    0                            0.0000    23933.0000              --- 
      0     0    18643.1466     1        0.0000    18643.1466        1     --- 

(18602.0, 18602)

# 2. Nonlinear Knapsack

### 2.1. Iterative Method

In [27]:
@numba.jit(nopython=True)
def nonlinear_knapsack_dp(a, p, b, t):
    m = len(a)
    f = zeros((m,b))
    alphas = zeros((m,b))
    
    cond = b*p[1]/float(p[1]+t)
    for alpha in range(b):
        if alpha == a[0] and alpha > cond:
            f[0][alpha] = 0
        elif alpha == a[0] and alpha <= cond:
            f[0][alpha] = p[0]-t*alpha/float(b-alpha)
            alphas[0][alpha] = alpha
        elif alpha != a[0]:
            f[0][alpha] = -sys.maxint

    for j in range(1,m):
        for alpha in range(b):
            res1=0; res2=0
            for ap_j in alphas[j-1]:
                if alpha==ap_j:    # if alpha in alphas[j-1]:
                    res1 = f[j-1][alpha]
                    alphas[j][alpha] = alpha
            for ap_j in alphas[j-1]:
                if alpha-a[j]==ap_j:    #if alpha-a[j] in alphas[j-1]:
                    res2 = f[j-1][alpha-a[j]]+p[j]-t*a[j]*b/float((b-alpha)**2+a[j]*(b-alpha))
                    alphas[j][alpha] = alpha
            f[j][alpha] = max(res1,res2)
    return f[-1]
#     return f[-1][-1] #########왜 안되지..
#     return f,alphas

In [28]:
def solve_nonlinear_knapsack_dp(prob, penalty):
    c = prob['c']
    a = prob['a']
    b = prob['b']
#     n = prob['n']
    optval = prob['optval']
    myVal = nonlinear_knapsack_dp(a,c,b,penalty)
    myVal = max(myVal)
    return myVal#, optval

In [29]:
# %%timeit
solve_nonlinear_knapsack_dp(probs[5],1)

8146.8717948717958

### 2.2. Cplex

In [30]:
def nonlinear_knapsack_cplex(n,a,b,c,t): #p(lnkp), t=penalty
    cplex = Cplex()
    
    # Set variables
    x = ['x%d' % i for i in range(n)]
    y = ['y%d' % i for i in range(n)]
    R = ['R']
    var_types = ['B']*n+['C']+['C']*n
    cplex.variables.add(obj=c+[-t]+[0]*n, types=var_types, names=x+R+y)  #obj=c+[-t], types=['B']*n + ['C'], names=x+R)
    
    # Set constraints
    M = 100000
    rows = [\
            [x, a],\
            [x+R+y, a+[-b]+a]\
            ]+\
            [[['y%d' %i,'R'],[1,-1]] for i in range(n)]+\
            [[['y%d' %i,'x%d' %i],[1,-M]] for i in range(n)]+\
            [[['x%d' %i,'R','y%d' %i],[M,1,-1]] for i in range(n)]
    
#     [SparsePair(ind=x+y, val=a+a),\
#      SparsePair()]
    
    cplex.linear_constraints.add(lin_expr=rows,senses=['L','E']+['L' for i in range(3*n)], \
                                 rhs=[b,0]+[0 for i in range(2*n)]+[M for i in range(n)], \
                                 names=['c%d'%(i+1) for i in range(3*n+2)])
    
    #Set sense
    cplex.objective.set_sense(cplex.objective.sense.maximize)
    
    #Solve
    cplex.solve()
    
    obj_val = cplex.solution.get_objective_value()
    x_res = [(cplex.solution.get_values(x[i]), x[i]) for i in range(n) \
             if cplex.solution.get_values(x[i]) > 0]
    
    cplex.write('test_nonlinear.lp') ###############
    return obj_val #, x_res

In [31]:
def solve_nonlinear_knapsack_cplex(p, t):
    n = p['n']
    a = p['a']
    b = p['b']
    c = p['c']
    return nonlinear_knapsack_cplex(n,a,b,c,t)

In [33]:
solve_nonlinear_knapsack_cplex(probs[5],1)

Found incumbent of value 0.000000 after 0.00 sec. (0.01 ticks)
Tried aggregator 1 time.
Reduced MIP has 152 rows, 101 columns, and 501 nonzeros.
Reduced MIP has 50 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.57 ticks)
Probing fixed 0 vars, tightened 1 bounds.
Probing time = 0.00 sec. (0.29 ticks)
Tried aggregator 1 time.
Reduced MIP has 152 rows, 101 columns, and 501 nonzeros.
Reduced MIP has 50 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.39 ticks)
Probing time = 0.00 sec. (0.32 ticks)
Clique table members: 12.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 4 threads.
Root relaxation solution time = 0.00 sec. (0.23 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0+    0                            0.0000    25697.0000              --- 
*  

8146.871794871795

### 2.3. Compare DP method and CPLEX method

In [34]:
def compare_dp_cplex(probs,pnt):  # pnt:penalty
    res = []
    for p in probs:
        dp_res = round(solve_nonlinear_knapsack_dp(p,pnt),2)
        cplex_res = round(solve_nonlinear_knapsack_cplex(p,pnt),2)
        if dp_res!=cplex_res: res.append(False)
        else: res.append(True)
    return res, dp_res, cplex_res

In [35]:
compare_dp_cplex(probs[2:3],1)

Found incumbent of value 0.000000 after 0.00 sec. (0.01 ticks)
Tried aggregator 1 time.
MIP Presolve modified 1 coefficients.
Reduced MIP has 152 rows, 101 columns, and 501 nonzeros.
Reduced MIP has 50 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.85 ticks)
Probing fixed 0 vars, tightened 1 bounds.
Probing time = 0.00 sec. (0.55 ticks)
Tried aggregator 1 time.
MIP Presolve modified 1 coefficients.
Reduced MIP has 152 rows, 101 columns, and 501 nonzeros.
Reduced MIP has 50 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.45 ticks)
Probing time = 0.00 sec. (0.56 ticks)
Clique table members: 23.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 4 threads.
Root relaxation solution time = 0.00 sec. (0.21 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0

([False], 5929.55, 5960.97)

# 3. Robust Knapsack

In [54]:
def robust_knapsack_cplex(c,d,a,n,b,gamma):
    cplex = Cplex()
    
    # Set variables
    x = ['x%d' % i for i in range(n)]
    y = ['y%d' % i for i in range(n)]
    p = ['pi']
    cplex.variables.add(obj=c+[0]*(n+1), types=['B']*n + ['C']*(n+1), names=x+y+p)
    
    # Set constraints
    row = [SparsePair(ind=x+y+p, val=a+[1]*n+[gamma])]+\
          [SparsePair(ind=[y[i]]+p+[x[i]], val=[1,1,-d[i]]) for i in range(n)]
        
        
    # Set constraints
#     row = [SparsePair(ind=x+y, val=a+[1]*n)]+\
#           [SparsePair(ind=[y[i]]+p+[x[i]], val=[1,1,-d[i]]) for i in range(n)]
        
    cplex.linear_constraints.add(lin_expr=row, senses=['L']+['G']*n, rhs=[b]+[0]*n)
    
    #Set sense
    cplex.objective.set_sense(cplex.objective.sense.maximize)
    
    #Solve
    cplex.solve()
    
    obj_val = cplex.solution.get_objective_value()
    x_res = [(cplex.solution.get_values(x[i]), x[i]) for i in range(n) \
             if cplex.solution.get_values(x[i]) > 0]
    return obj_val #, x_res

In [55]:
def solve_robust_knapsack_cplex(p, gamma):
    n = p['n']
    a = p['a']
    b = p['b']
    c = p['c']
    d = [ai*0.3 for ai in a]
    return robust_knapsack_cplex(c,d,a,n,b,gamma)

In [56]:
solve_robust_knapsack_cplex(probs[0],20)

Found incumbent of value 0.000000 after 0.00 sec. (0.01 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 12 rows and 24 columns.
Reduced MIP has 39 rows, 77 columns, and 191 nonzeros.
Reduced MIP has 38 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.26 ticks)
Probing time = 0.00 sec. (0.43 ticks)
Tried aggregator 1 time.
Reduced MIP has 39 rows, 77 columns, and 191 nonzeros.
Reduced MIP has 38 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.14 ticks)
Probing time = 0.00 sec. (0.28 ticks)
Clique table members: 133.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 4 threads.
Root relaxation solution time = 0.00 sec. (0.08 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0+    0                            0.0000    21292.0000              --- 


7278.0

# 4. integer knapsack

In [36]:
def h(lmd, c, a):
    if lmd<=0:return 0
    val = max([c[j]+h(lmd-a[j], c, a) for j in range(1,len(c)) if lmd>=a[j]])
    return max(0,val)

In [37]:
def h2(r,lmd,c,a):
    pick = c[r]
    if lmd<=0 or r<=0: return 0
    if lmd<a[r]:
        pick=0
    return max(h2(r-1,lmd,c,a), pick+h2(r,lmd-a[r],c,a))

In [38]:
c = [0,7,9,2,15]
a = [0,3,4,1,7]

print h(10, c, a)
print h2(4, 10, c, a)

23
23
